In [4]:
record_2017[0:10]

[[1, 24, 297, 1],
 [1, 30, 298, 1],
 [1, 25, 299, 1],
 [1, 32, 303, 1],
 [1, 31, 304, 1],
 [1, 40, 304, 1],
 [1, 33, 305, 1],
 [1, 28, 308, 1],
 [1, 27, 309, 1],
 [1, 28, 311, 1]]

In [ ]:
import numpy as np
from tkinter import *
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
import math
# Import pandas as a alias 'pd'
import pandas as pd

# Load the CSV files "marathon_results_2015 ~ 2017.csv" under "data" folder
marathon_2015_2017_qualifying = pd.read_csv("./data/marathon_2015_2017_qualifying.csv")

# Merge 2015, 2016 and 2017 files into marathon_2015_2017 file index by Official Time
# record = pd.DataFrame(marathon_2015_2017,columns=['5K',  '10K',  '15K',  '20K', 'Half',  '25K',  '30K',  '35K',  '40K',  'Official Time']).sort_values(by=['Official Time'])
marathon_2015_2016 = marathon_2015_2017_qualifying[marathon_2015_2017_qualifying['Year'] != 2017]
marathon_2017 = marathon_2015_2017_qualifying[marathon_2015_2017_qualifying['Year'] == 2017]

df_2015_2016 = pd.DataFrame(marathon_2015_2016,columns=['M/F',  'Age',  'Pace',  'qualifying'])
df_2017 = pd.DataFrame(marathon_2017,columns=['M/F',  'Age',  'Pace',  'qualifying'])

# Dataframe to List
record_2015_2016 = df_2015_2016.values.tolist()
record_2017 = df_2017.values.tolist()

gender_list = ['Female', 'Male']
grad_fig = Figure(figsize=(10, 6), dpi=100)
grad_ax = grad_fig.add_subplot(111)
grad_ax.set_xlim(15, 88)
grad_ax.set_ylim(0, 1300)
grad_ax.set_ylabel("Pace : Runner's overall minute per mile pace")
grad_ax.set_xlabel("Age : Age on race day")

def seconds_to_hhmmss(seconds):
    hours = seconds // (60*60)
    seconds %= (60*60)
    minutes = seconds // 60
    seconds %= 60
    return "%02i:%02i:%02i" % (hours, minutes, seconds)

def normalization(record):
    r0 = record[0]
    r1 = record[1] / 10
    r2 = record[2] / 100
    return [r0, r1, r2]

# X and Y data from 0km to 30km    
# x_train = [ r[0:3] for r in record_2015_2016]
x_train = [ normalization(r[0:3]) for r in record_2015_2016]
y_train = [ [r[-1]] for r in record_2015_2016]
# x_test = [ r[0:3] for r in record_2017]
x_test = [ r[0:3] for r in record_2017]
y_test = [ [r[-1]] for r in record_2017]

def histogram():
    t_a = int(t_aSpbox.get()) - 1
    runner = x_test[t_a]
    print(runner)
    t_g = int(runner[0])
    t_y = int(runner[1])
    t_p = int(runner[2])
    if(t_g):
        gender_color = 'b'
    else:
        gender_color = 'r'  
    gender_record = df_2017[df_2017['M/F'] == t_g]
    gender_age_record = gender_record[gender_record.Age == t_y] 
    gender_age_record_list = gender_age_record.values.tolist() 
    
    grad_ax.plot(gender_record.Age, gender_record.Pace, '.', color=gender_color, alpha=0.5)
    grad_ax.plot(t_y, t_p, 'yd')
    stat = gender_age_record['Pace'].describe()
    print(stat)
    title = 'Gender : '+gender_list[t_g]+', Age : '+str(t_y)+', Pace : '+str(t_p)
    grad_ax.set_title(title)
    grad_ax.annotate('['+gender_list[t_g]+', '+str(t_y)+']', (75, 1200), fontsize=10)
    grad_ax.annotate("%10s %7i" % ('Count : ', stat[0]), (75, 1150), fontsize=10)
    grad_ax.annotate("%10s %7.3f" % ('Mean :  ', stat[1]), (75, 1100), fontsize=10)
    grad_ax.annotate("%10s %7.3f" % ('25% :   ', stat[3]), (75, 1050), fontsize=10)
    grad_ax.annotate("%10s %7.3f" % ('75% :   ', stat[5]), (75, 1000), fontsize=10)
        
    grad_fig.canvas.draw()    
    
def learing(): 
    """
    MAchine Learning, Tensorflow2 
    """
    # Tensorflow2 Linear Regression
    import tensorflow as tf
    import numpy as np
    
    t_a = int(t_aSpbox.get()) - 1
    runner = x_test[t_a]
    t_g = int(runner[0])#성별
    t_y = int(runner[1])#나이
    t_p = int(runner[2])#페이스

    t_t = int(t_tSpbox.get())#트레이닝 개수
    t_r = float(t_rSpbox.get())#러닝레이트

    # Define Sequential model and Dense
    model =  tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(1, input_shape=(3,), activation='sigmoid'))
    # Use softmax  
    # model.add(Activation('sigmoid'))
    # Stochastic gradient descent (SGD) Optimizer
    sgd = tf.keras.optimizers.SGD(lr=t_r)
    # categorical_crossentropy loss function
    model.compile(loss='binary_crossentropy', 
                  optimizer=sgd,
                  metrics=['accuracy'])

    # prints summary of the model to the terminal
    model.summary()

    # Train the model
    history = model.fit(np.array(x_train), np.array(y_train), epochs=t_t)

    # Fit the line
    log_ScrolledText.insert(END, '\nGender :'+gender_list[t_g]+', Age :'+str(t_y)+', Pace :'+str(t_p)+'\n', 'TITLE')
    log_ScrolledText.insert(END, '\n\nCost Decent\n\n','HEADER')
    log_ScrolledText.insert(END, "%10s %20s %20s" % ('Step', 'Cost', 'Accuracy(%)')+'\n\n')
    for step in range(t_t):
        if step % 100 == 0:
            cost_val = history.history['loss'][step]
            a_val = history.history['accuracy'][step]
            log_ScrolledText.insert(END, "%10i %20.5f %20.7f" % (step, cost_val, a_val*100)+'\n')
    
    winner = [ t_g, t_y, t_p ]
    result = model.predict(np.array([normalization(winner)]))

    log_ScrolledText.insert(END, '\n\n')
    log_ScrolledText.insert(END, "%10s %20s" % ('Value        ', 'Qualifying Prediction\n\n'), 'HEADER')
    if(result[0] > 0.5):
        log_ScrolledText.insert(END, "%10.7f %20s" % (result[0], 'Qualifier\n\n'), 'Qualifier')
    else:
        log_ScrolledText.insert(END, "%10.7f %20s" % (result[0], 'DisQualifier\n\n'), 'DisQualifier')        
    
#main
main = Tk()
main.title("Logistic Regression Binary Classification")
main.geometry()

label=Label(main, text='Logistic Regression Binary Classification')
label.config(font=("Courier", 18))
label.grid(row=0,column=0,columnspan=6)

t_aVal  = IntVar(value=1)
t_aSpbox = Spinbox(main, textvariable=t_aVal ,from_=0, to=len(x_test), increment=1, justify=RIGHT)
#t_aSpbox.config(state='readonly')
t_aSpbox.grid(row=1,column=1)
t_aLabel=Label(main, text='Rank of runner : ')                
t_aLabel.grid(row=1,column=0)

t_tVal  = IntVar(value=10000)
t_tSpbox = Spinbox(main, textvariable=t_tVal ,from_=0, to=100000, increment=1000, justify=RIGHT)
#t_tSpbox.config(state='readonly')
t_tSpbox.grid(row=1,column=3)
t_tLabel=Label(main, text='Number of train : ')                
t_tLabel.grid(row=1,column=2)

t_rVal  = DoubleVar(value=1e-2)
t_rSpbox = Spinbox(main, textvariable=t_rVal ,from_=0, to=1, increment=1e-2, justify=RIGHT)
#t_rSpbox.config(state='readonly')
t_rSpbox.grid(row=1,column=5)
t_rLabel=Label(main, text='Learning rate : ')                
t_rLabel.grid(row=1,column=4)

Button(main,text="Histogram", height=2,command=lambda:histogram()).grid(row=2, column=0, columnspan=3, sticky=(W, E))
Button(main,text="Prediction", height=2,command=lambda:learing()).grid(row=2, column=3, columnspan=3, sticky=(W, E))

grad_canvas = FigureCanvasTkAgg(grad_fig, main)
grad_canvas.get_tk_widget().grid(row=3,column=0,columnspan=6)

log_ScrolledText = tkst.ScrolledText(main, height=15)
log_ScrolledText.grid(row=4,column=0,columnspan=6, sticky=(N, S, W, E))
log_ScrolledText.configure(font='TkFixedFont')
log_ScrolledText.tag_config('Qualifier', foreground='blue', font=("Helvetica", 16))
log_ScrolledText.tag_config('DisQualifier', foreground='red', font=("Helvetica", 16))
log_ScrolledText.tag_config('HEADER', foreground='gray', font=("Helvetica", 14), underline=1)
log_ScrolledText.tag_config('TITLE', foreground='orange', font=("Helvetica", 18), underline=1, justify='center')

main.mainloop()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Train on 53228 samples
Epoch 1/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 1.9636 - accuracy: 0.6898
Epoch 2/1000
53228/53228 [==============================] - 3s 55us/sample - loss: 0.8689 - accuracy: 0.7920
Epoch 3/1000
53228/53228 [==============================] - 3s 54us/sample - loss: 0.5919 - accuracy: 0.8317
Epoch 4/1000
53228/53228 [==============================] - 3s 54us/sample - loss: 0.4368 - accuracy: 0.8607
Epoch 5/1000
53228/53228 [==============================] - 3s 54us/sample - loss: 0.4227 - accuracy: 0.8676
Epoch 6/1000
53228/53228 [==============================] - 3s 54us/sample - loss: 0.3

53228/53228 [==============================] - 3s 53us/sample - loss: 0.2909 - accuracy: 0.9028
Epoch 72/1000
53228/53228 [==============================] - 3s 52us/sample - loss: 0.2702 - accuracy: 0.9072
Epoch 73/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2795 - accuracy: 0.9053
Epoch 74/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2867 - accuracy: 0.9035
Epoch 75/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2875 - accuracy: 0.9040
Epoch 76/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2864 - accuracy: 0.9022
Epoch 77/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2937 - accuracy: 0.9020
Epoch 78/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2803 - accuracy: 0.9047
Epoch 79/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2831 - accuracy: 0.9053
Epoch 80/1000
53228/5322

53228/53228 [==============================] - 3s 53us/sample - loss: 0.2784 - accuracy: 0.9043
Epoch 145/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2914 - accuracy: 0.9015
Epoch 146/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2782 - accuracy: 0.9033
Epoch 147/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2853 - accuracy: 0.9047
Epoch 148/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2847 - accuracy: 0.9041
Epoch 149/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2842 - accuracy: 0.9062
Epoch 150/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2902 - accuracy: 0.9014
Epoch 151/1000
53228/53228 [==============================] - 3s 55us/sample - loss: 0.2836 - accuracy: 0.9040
Epoch 152/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2858 - accuracy: 0.9035
Epoch 153/1000
5

Epoch 218/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2746 - accuracy: 0.9058
Epoch 219/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2860 - accuracy: 0.9019
Epoch 220/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2843 - accuracy: 0.9052
Epoch 221/1000
53228/53228 [==============================] - 3s 57us/sample - loss: 0.2819 - accuracy: 0.9027
Epoch 222/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2917 - accuracy: 0.9014
Epoch 223/1000
53228/53228 [==============================] - 3s 57us/sample - loss: 0.2856 - accuracy: 0.9037
Epoch 224/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2875 - accuracy: 0.9048
Epoch 225/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2733 - accuracy: 0.9062
Epoch 226/1000
53228/53228 [==============================] - 3s 57us/sample - loss: 0.2829 - accuracy: 0.9028
E

53228/53228 [==============================] - 3s 66us/sample - loss: 0.2808 - accuracy: 0.9048
Epoch 292/1000
53228/53228 [==============================] - 3s 57us/sample - loss: 0.2914 - accuracy: 0.9024
Epoch 293/1000
53228/53228 [==============================] - 3s 55us/sample - loss: 0.2876 - accuracy: 0.9018
Epoch 294/1000
53228/53228 [==============================] - 3s 57us/sample - loss: 0.2935 - accuracy: 0.9021
Epoch 295/1000
53228/53228 [==============================] - 3s 55us/sample - loss: 0.2840 - accuracy: 0.9036
Epoch 296/1000
53228/53228 [==============================] - 3s 56us/sample - loss: 0.2770 - accuracy: 0.9047
Epoch 297/1000
53228/53228 [==============================] - 3s 54us/sample - loss: 0.2929 - accuracy: 0.9030
Epoch 298/1000
53228/53228 [==============================] - 3s 53us/sample - loss: 0.2743 - accuracy: 0.9055
Epoch 299/1000
53228/53228 [==============================] - 3s 54us/sample - loss: 0.2852 - accuracy: 0.9047
Epoch 300/1000
5

Epoch 365/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2887 - accuracy: 0.9029
Epoch 366/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2877 - accuracy: 0.9017
Epoch 367/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2793 - accuracy: 0.9044
Epoch 368/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2883 - accuracy: 0.9033
Epoch 369/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2718 - accuracy: 0.9042
Epoch 370/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2833 - accuracy: 0.9044
Epoch 371/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2865 - accuracy: 0.9028
Epoch 372/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2890 - accuracy: 0.9028
Epoch 373/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2933 - accuracy: 0.9021
E

53228/53228 [==============================] - 3s 63us/sample - loss: 0.2839 - accuracy: 0.9034
Epoch 439/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2888 - accuracy: 0.9016
Epoch 440/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2864 - accuracy: 0.9017
Epoch 441/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2880 - accuracy: 0.9034
Epoch 442/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2871 - accuracy: 0.9022
Epoch 443/1000
53228/53228 [==============================] - 3s 57us/sample - loss: 0.2969 - accuracy: 0.9014
Epoch 444/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2845 - accuracy: 0.9050
Epoch 445/1000
53228/53228 [==============================] - 3s 58us/sample - loss: 0.2921 - accuracy: 0.9003
Epoch 446/1000
53228/53228 [==============================] - 3s 57us/sample - loss: 0.2888 - accuracy: 0.9023
Epoch 447/1000
5

Epoch 512/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2848 - accuracy: 0.9032
Epoch 513/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2781 - accuracy: 0.9041
Epoch 514/1000
53228/53228 [==============================] - 3s 59us/sample - loss: 0.2903 - accuracy: 0.9012
Epoch 515/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2905 - accuracy: 0.9026
Epoch 516/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2853 - accuracy: 0.9033
Epoch 517/1000
53228/53228 [==============================] - 3s 61us/sample - loss: 0.2853 - accuracy: 0.9028
Epoch 518/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2976 - accuracy: 0.9017
Epoch 519/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2892 - accuracy: 0.9023
Epoch 520/1000
53228/53228 [==============================] - 3s 61us/sample - loss: 0.2881 - accuracy: 0.9033
E

53228/53228 [==============================] - 3s 62us/sample - loss: 0.2792 - accuracy: 0.9048
Epoch 586/1000
53228/53228 [==============================] - 3s 61us/sample - loss: 0.2842 - accuracy: 0.9037
Epoch 587/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2838 - accuracy: 0.9048
Epoch 588/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2823 - accuracy: 0.9042
Epoch 589/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2776 - accuracy: 0.9058
Epoch 590/1000
53228/53228 [==============================] - 3s 60us/sample - loss: 0.2836 - accuracy: 0.9048
Epoch 591/1000
53228/53228 [==============================] - 3s 62us/sample - loss: 0.2771 - accuracy: 0.9054
Epoch 592/1000
53228/53228 [==============================] - 3s 62us/sample - loss: 0.2929 - accuracy: 0.9030
Epoch 593/1000
53228/53228 [==============================] - 3s 61us/sample - loss: 0.2823 - accuracy: 0.9040
Epoch 594/1000
5